In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from __future__ import annotations

from tqdm.notebook import tqdm
from collections import OrderedDict
import glob
import cv2
import random
import time
import copy

from torchvision.io import read_image
import matplotlib.pyplot as plt

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision.models import resnet34, ResNet34_Weights
from torch.optim import lr_scheduler

# Fix random seed

In [ ]:
def fix_seed(seed):
    # random
    random.seed(seed)
#     # Numpy
#     np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
#     # Tensorflow
#     tf.random.set_seed(seed)

SEED = 3407
fix_seed(SEED)

# Load dataset

In [ ]:
# Train data
train_target = pd.read_csv('/kaggle/input/aptos2019-blindness-detection/train.csv')

# train_target['diagnosis'].value_counts().plot(kind='bar');
# plt.title('Class counts');

In [ ]:
# paths = glob.glob(r'/kaggle/input/aptos2019-blindness-detection/train_images/*.png')
# widths = []
# heights = []

# for path in tqdm(paths):
#     img = cv2.imread(path)
#     h, w = img.shape[:2]
    
#     widths.append(w)
#     heights.append(h)
    
# heights, widths = zip(*[cv2.imread(path).shape[:2] for path in tqdm(paths)])

In [ ]:
# plt.hist(heights, bins = 10)
# plt.title('heights')
# plt.show()

In [ ]:
# plt.hist(widths, bins = 10)
# plt.title('widths')
# plt.show()

# Define dataset and dataloader

In [ ]:
# # Preprocess images
resize_h = 224
resize_w = 224
input_shape = (resize_h, resize_w)
# # train_images = list()
# # for path in paths:
# #     img = cv2.imread(path)
# #     img = cv2.resize(img, dsize=(resize_h, resize_w))
# #     train_images.append(img)
print(resize_h, resize_w)

In [ ]:
# Define dataset class

class CustomImageDataset(Dataset):
    def __init__(self, img_labels: pd.DataFrame, img_dir, transform=None, target_transform=None, train=True):
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.img_labels = img_labels

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0] + ".png")
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
# pre_transforms = transforms.Compose([
#     transforms.ToPILImage(),
#     transforms.ToTensor(),
#     transforms.Resize(input_shape)
# ])

# tensor_aptos = CustomImageDataset(train_target, "/kaggle/input/aptos2019-blindness-detection/train_images", pre_transforms)


# imgs = torch.stack([img_t for img_t, _ in tqdm(tensor_aptos)], dim=3)
# imgs.shape

In [ ]:
# mean = imgs.view(3, -1).mean(dim=1)
mean = (0.4138, 0.2210, 0.0737)
print(mean)

In [ ]:
# std = imgs.view(3, -1).std(dim=1)
std = (0.2745, 0.1499, 0.0808)
print(std)

In [ ]:
input_shape = (resize_h, resize_w)
        
data_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Resize(input_shape),
    transforms.Normalize(mean, std)
])

trainval_data = CustomImageDataset(train_target, "/kaggle/input/aptos2019-blindness-detection/train_images", data_transforms)
# test_data = CustomImageDataset(test_target, "/kaggle/input/aptos2019-blindness-detection/test_images", data_transforms)

In [ ]:
# labels_map = {
#     0: "No",
#     1: "Mi",
#     2: "Mo",
#     3: "Se",
#     4: "Pr"
# }

In [ ]:
# def display_images(dataset):
#     figure = plt.figure(figsize=(10, 10))
#     cols, rows = 10, 10
#     for i in range(1, cols * rows + 1):
#         sample_idx = torch.randint(len(dataset), size=(1,)).item()
#         img, label = trainval_data[sample_idx]
#         figure.add_subplot(rows, cols, i)
#         plt.title(labels_map[label])
#         plt.axis("off")
#         plt.imshow(img.squeeze().permute(1,2,0))
#     plt.show()

In [ ]:
# TrainVal images
# display_images(trainval_data)

In [ ]:
# Test images
# display_images(test_data)

In [ ]:
# Define dataloader

batch_size = 64

val_size = round(len(trainval_data) * 0.2)
train_size = len(trainval_data) - val_size
train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_trainval = torch.utils.data.DataLoader(
    trainval_data,
    batch_size=batch_size,
    shuffle=True
)

dataloaders_dict = {'Train': dataloader_train, 'Validation': dataloader_valid}
dataloader_dict = {'TrainVal': dataloader_trainval}

# Define CNN model

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# modified code from https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
def set_parameter_requires_grad(model, train):
    for param in model.parameters():
        param.requires_grad = train

In [ ]:
# model = torchvision.models.resnet34(weights='IMAGENET1K_V1') # resnet34 is possibly better than resnet50
model = torchvision.models.resnet34(weights=ResNet34_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(model.fc.in_features, 5)

# optimizer = optim.Adam(model.parameters(), lr=1e-3) # according to an article
optimizer = optim.SGD(model.parameters(),lr=1e-3,momentum=0.9, weight_decay=0.0001) # fix this
criterion = nn.CrossEntropyLoss() # according to an article
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

# Define early stopping

In [ ]:
# Code from https://github.com/Bjarten/early-stopping-pytorch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
        self.epoch = 0
        self.best_epoch = 0
    def __call__(self, val_loss, model):

        score = -val_loss
        self.epoch += 1

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
            self.best_epoch = self.epoch

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

# Define train model

In [ ]:
def train_model(model, device, dataloaders: dict, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()
    
    model = model.to(device)
    
    histories = {'Accuracy': {phase: list() for phase in dataloaders.keys()}, 'Loss': {phase: list() for phase in dataloaders.keys()}}
    
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)
    early_stopping = EarlyStopping(patience=10, verbose=True)
    best_epoch = 0
    
    terminate = False

    for epoch in range(num_epochs):
        print('EPOCH: {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in dataloaders.keys():
            if phase == 'Validation':
                model.eval()   # Set model to evaluate mode
            else:
                model.train()  # Set model to training mode
            
            losses = []
            num = 0
            true_num = 0

            # Iterate over data.
            for x, t in tqdm(dataloaders[phase]):
                model.zero_grad()  # Initialise gradient descent
                x, t = x.to(device), t.to(device)

                # zero the parameter gradients
                # optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'Train' or phase == 'TrainVal'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and (phase == 'Train' or phase == 'TrainVal'):
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        y, aux_outputs = model(x)
                        loss1 = criterion(y, t)
                        loss2 = criterion(aux_outputs, t)
                        loss = loss1 + 0.4*loss2
                    else: # valid
                        y = model(x)  # Forward propagation
                        loss = criterion(y, t)

                    pred = y.argmax(dim=1)  # 最大値を取るラベルを予測ラベルとする

                    # backward + optimize only if in training phase
                    if (phase == 'Train' or phase == 'TrainVal'):
                        loss.backward()
                        optimizer.step()
                        # scheduler.step()
                        
                    losses.append(loss.tolist())

                    acc = torch.where(t.to("cpu") - pred.to("cpu") == 0, torch.ones_like(t).to("cpu"), torch.zeros_like(t).to("cpu"))
                    num += acc.size()[0]
                    true_num += acc.sum().item()
                    
            epoch_loss = np.mean(losses)
            epoch_acc = true_num / num
            
            histories['Loss'][phase].append(epoch_loss)
            histories['Accuracy'][phase].append(epoch_acc)
            

            print('{} [Loss: {:.4f}, Accuracy: {:.4f}]'.format(phase, epoch_loss, epoch_acc))
            print()
            if phase == 'Validation':
                early_stopping(np.mean(losses), model) # 最良モデルならモデルパラメータ保存 save hyper parameters if best
                if early_stopping.early_stop: 
                    # 一定epochだけval_lossが最低値を更新しなかった場合、ここに入り学習を終了
                    num_epochs = epoch + 1
                    best_epoch = early_stopping.best_epoch
                    terminate = True
                    break
            else:
                scheduler.step()
        
        if terminate:
            break
        print()
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    return model, num_epochs, best_epoch, histories

In [ ]:
def plot(n_epochs, histories: dict):
    epochs = np.arange(1, n_epochs + 1)

    fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(8, 3))

    for ax, metric in zip([ax1, ax2], histories.keys()):
        ax.set_title(metric)
        for key in histories[metric].keys():
            ax.plot(epochs, histories[metric][key], label=key)
        ax.set_xlabel("Epoch")
        ax.legend()

    plt.show()

# Save the initial model

In [ ]:
torch.save(model.state_dict(), 'initial_weight.pth')

# Train the model

In [ ]:
# Fine tuning
# set_parameter_requires_grad(model, False)
# model, num_epochs, best_epoch, histories = train_model(model, device, dataloaders_dict, criterion, optimizer, exp_lr_scheduler, num_epochs=100, is_inception=False)

In [ ]:
# plot(num_epochs, histories)

In [ ]:
# torch.save(model.state_dict(), 'fine_tuned.pth')

In [ ]:
# set_parameter_requires_grad(model, True)
model, num_epochs, best_epoch, histories = train_model(model, device, dataloaders_dict, criterion, optimizer, num_epochs=100, is_inception=False)

In [ ]:
plot(num_epochs, histories)

# Reset the model

In [ ]:
model.load_state_dict(torch.load('initial_weight.pth'))

In [ ]:
model, num_epochs, best_epoch, histories = train_model(model, device, dataloader_dict, criterion, optimizer, num_epochs=best_epoch, is_inception=False)

In [ ]:
plot(num_epochs, histories)

# Save the model

In [ ]:
torch.save(model, "resnet34.pth") # save the entire model with weight parameters, trained with train data